# 10-1. 프로젝트 : 커스텀 프로젝트 직접 만들기
실습 코드에서 수행해 본 내용을 토대로, 이번에는 한국어 데이터셋에 도전해보겠습니다.

앞서 본 GLUE benchmark의 한국어 버전 KLUE benchmark를 들어보신 적 있나요?

https://klue-benchmark.com/

GLUE와 마찬가지로 한국어 자연어처리에 대한 이해도를 높이기 위해 만들어진 데이터셋 benchmark입니다. 총 8가지의 데이터셋이 있습니다. 다만 이번 시간에 진행할 프로젝트는 KLUE의 dataset을 활용하는 것이 아닌, model(klue/ber-base)를 활용하여 NSMC(Naver Sentiment Movie Corpus) task를 도전해보겠습니다.

모델과 데이터에 관한 정보는 링크를 참조해주세요.

KLUE/Bert-base - https://huggingface.co/klue/bert-base

NSMC - https://github.com/e9t/nsmc

준비가 되셨다면 아래와 같은 순서로 진행해주세요.

### 라이브러리 버전을 확인해 봅니다.
사용할 라이브러리 버전을 둘러봅시다.

In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


# STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

https://huggingface.co/datasets

### Huggingface transformers 설치 및 환경 구성

In [2]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

### nsmc 데이터 불러오기 및 살펴보기

In [3]:
import datasets
from datasets import load_dataset

huggingface_nsmc_dataset = load_dataset('nsmc')
print(huggingface_nsmc_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [4]:
train = huggingface_nsmc_dataset['train']
train

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [5]:
cols = train.column_names
cols

['id', 'document', 'label']

In [6]:
train['document'][:100]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.',
 '원작의 긴장감을 제대로 살려내지못했다.',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?',
 '걍인피니트가짱이다.진짜짱이다♥',
 '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~',
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해',
 '담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.',
 '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지',
 'ㄱ냥 매번 긴장되고 재밋음ㅠㅠ',
 '참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다',
 '굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐',
 '이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드!!♥',
 '약탈자를 위한 변명, 이라. 저놈들은 착한놈들 절대 아닌걸요.',
 '나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님',
 '보면서 웃지 않는 건 불가능하다',
 '재미없다 지루하고. 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남....바베트의 만찬은 이야기도 있

In [7]:
train['label'][:10]

[0, 1, 0, 0, 1, 0, 0, 0, 1, 1]

In [8]:
train['id'][:10]

['9976970',
 '3819312',
 '10265843',
 '9045019',
 '6483659',
 '5403919',
 '7797314',
 '9443947',
 '7156791',
 '5912145']

In [9]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




# STEP 2. klue/bert-base model 및 tokenizer 불러오기

In [65]:
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

# distilbert-base-uncased

#huggingface_model = AutoModel.from_pretrained("klue/bert-base")
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels = 2)
#huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-small', num_labels = 2)
huggingface_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

# STEP 3.  tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [66]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = True,
        max_length=41, # nsmc 데이터 93%가 lenth 41이내
        return_token_type_ids = False,
        )

In [67]:
hf_dataset = huggingface_nsmc_dataset.map(transform, batched=True)

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-aa04a674437bedfa.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-a0e1585694cf16af.arrow


In [68]:
hf_dataset['train']['document'][:10]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.',
 '원작의 긴장감을 제대로 살려내지못했다.',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?']

In [69]:
len(hf_dataset['train']['document'])

150000

In [70]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 50000
    })
})

### validation set 생성하기

In [71]:
# 1. Dateset split 적용
spilt_dataset = hf_dataset['train'].train_test_split(test_size=0.2)
spilt_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 30000
    })
})

In [72]:
# 2. spilt한 데이터 저장
train_dataset, validation_dataset = spilt_dataset['train'], spilt_dataset['test']
train_dataset, validation_dataset

(Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 120000
 }),
 Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 30000
 }))

In [73]:
# 3. train & validation & test split.
hf_train_dataset = train_dataset.shuffle(seed=42).select(range(5000))
hf_val_dataset = validation_dataset.shuffle(seed=42).select(range(1000))
hf_test_dataset = hf_dataset['test'].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-7267e811ee508baf.arrow


In [74]:
hf_train_dataset, hf_val_dataset, hf_test_dataset

(Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 5000
 }),
 Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 1000
 }),
 Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 1000
 }))

### ㄴ 문제해결회고 
: 과도한 학습 시간(8시간 30분)으로 인하여 데이터 샘플링 수행

### Trainer를 활용한 학습

In [75]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/Natural_Language_Processing/10_huggingface_project/transformers'

training_arguments = TrainingArguments(
    output_dir,                            # output이 저장될 경로
    evaluation_strategy="epoch",           # evaluation하는 빈도
    learning_rate = 2e-5,                  # learning_rate
    per_device_train_batch_size = 8,       # 각 device 당 batch size
    per_device_eval_batch_size = 8,        # evaluation 시에 batch size
    num_train_epochs = 3,                 # train 시킬 총 epochs
    weight_decay = 0.01,                   # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [76]:
# from datasets import load_metric
# from sklearn.metrics import accuracy_score, f1_score

# # 정확도(Accuracy)와 F1 점수(F1 Score)를 계산하는 compute_metrics 함수
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     acc = accuracy_score(labels, preds)
#     f1 = f1_score(labels, preds, average='weighted')
#     return {
#         'accuracy': acc,
#         'f1': f1,
#     }

In [77]:
# glue/mrpc의 acc, f1 지표 활용
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Trainer에 model, arguments, train_dataset, eval_dataset, compute_metrics를 넣고 train을 진행합니다.

In [78]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.400400,0.431659,0.847000,0.840459
2,0.279900,0.659398,0.834000,0.832998
3,0.185800,0.790909,0.838000,0.841176


Saving model checkpoint to /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-1000/pytorch_model.

TrainOutput(global_step=1875, training_loss=0.2597946756998698, metrics={'train_runtime': 256.4822, 'train_samples_per_second': 58.484, 'train_steps_per_second': 7.31, 'total_flos': 316041599700000.0, 'train_loss': 0.2597946756998698, 'epoch': 3.0})

마지막으로 test 데이터셋으로 평가를 합니다.

In [79]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.7093942761421204,
 'eval_accuracy': 0.854,
 'eval_f1': 0.8522267206477733,
 'eval_runtime': 3.061,
 'eval_samples_per_second': 326.695,
 'eval_steps_per_second': 40.837,
 'epoch': 3.0}

### 결과
: 다양한 시도를 했을 때 베이스라인 모델의 최고 validation accuracy는 0.888로 나타났다.

# STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

In [25]:
#메모리를 비워줍니다.
del huggingface_model

### monologg/koelectra-base-v3 모델을 활용한 fine-tuning 수행

In [31]:
from transformers import ElectraModel, ElectraTokenizer, ElectraForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification

# monologg/koelectra-base-v3-discriminator
# huggingface_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# huggingface_model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels = 2)

# monologg/koelectra-base-finetuned-sentiment
huggingface_tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-finetuned-sentiment")
huggingface_model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-finetuned-sentiment", num_labels = 2)

# monologg/koelectra-base-discriminator
# huggingface_tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
# huggingface_model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-discriminator", num_labels = 2)

https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /aiffel/.cache/huggingface/transformers/tmp4z3icby0


Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

storing https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/tokenizer_config.json in cache at /aiffel/.cache/huggingface/transformers/6c420e1e4e8a4bb8cef587982266920738d38d77d6210cc0fa20f4e50e24f04c.767d1b760a83978bae6c324157fad57ee513af333a7cea6986e852579f6f0dd1
creating metadata file for /aiffel/.cache/huggingface/transformers/6c420e1e4e8a4bb8cef587982266920738d38d77d6210cc0fa20f4e50e24f04c.767d1b760a83978bae6c324157fad57ee513af333a7cea6986e852579f6f0dd1
https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/config.json not found in cache or force_download set to True, downloading to /aiffel/.cache/huggingface/transformers/tmpt8naqg02


Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

storing https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/config.json in cache at /aiffel/.cache/huggingface/transformers/a0a010ec774eac100a824caba3e4730b7d8d74824d5fd9a63fbf3e910354d3f8.235694219f1bd11aa2574b58d596fa99ecdcab54e7cd4eb291e2186b33bd124b
creating metadata file for /aiffel/.cache/huggingface/transformers/a0a010ec774eac100a824caba3e4730b7d8d74824d5fd9a63fbf3e910354d3f8.235694219f1bd11aa2574b58d596fa99ecdcab54e7cd4eb291e2186b33bd124b
loading configuration file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a0a010ec774eac100a824caba3e4730b7d8d74824d5fd9a63fbf3e910354d3f8.235694219f1bd11aa2574b58d596fa99ecdcab54e7cd4eb291e2186b33bd124b
Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_

Downloading:   0%|          | 0.00/273k [00:00<?, ?B/s]

storing https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/vocab.txt in cache at /aiffel/.cache/huggingface/transformers/bfe61bb60a4fddad9575ec468f02c708afd9458e51599f876b2164ae6870f128.42b747a439e42193b1b54614a2e0c4608d8ec136b7405c799d29f667ccdbb27d
creating metadata file for /aiffel/.cache/huggingface/transformers/bfe61bb60a4fddad9575ec468f02c708afd9458e51599f876b2164ae6870f128.42b747a439e42193b1b54614a2e0c4608d8ec136b7405c799d29f667ccdbb27d
loading file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/bfe61bb60a4fddad9575ec468f02c708afd9458e51599f876b2164ae6870f128.42b747a439e42193b1b54614a2e0c4608d8ec136b7405c799d29f667ccdbb27d
loading file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/added_tokens.jso

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

storing https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/pytorch_model.bin in cache at /aiffel/.cache/huggingface/transformers/8a0cc1df8fdcf3fba5608bcd23344e15a1a41fff3b41ead3ff1e9fe22a0e78c0.a42adf6aa236918d3389c3e8eceaad08f1354410aa9f353cb469fd0472e6e426
creating metadata file for /aiffel/.cache/huggingface/transformers/8a0cc1df8fdcf3fba5608bcd23344e15a1a41fff3b41ead3ff1e9fe22a0e78c0.a42adf6aa236918d3389c3e8eceaad08f1354410aa9f353cb469fd0472e6e426
loading weights file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transformers/8a0cc1df8fdcf3fba5608bcd23344e15a1a41fff3b41ead3ff1e9fe22a0e78c0.a42adf6aa236918d3389c3e8eceaad08f1354410aa9f353cb469fd0472e6e426
Some weights of the model checkpoint at monologg/koelectra-base-finetuned-sentiment were not used when initializing ElectraForSequenceClassification: ['classifier.bias', 'classifier.weight']
- This IS expected if

### monologg/koelectra-base-v3 토크나이져 활용

In [32]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        #max_length=41, # nsmc 데이터 93%가 lenth 41이내
        return_token_type_ids = False,
        )

In [33]:
hf_dataset = huggingface_nsmc_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

### validation set 생성하기

In [34]:
# 1. Dateset split 적용
spilt_dataset = hf_dataset['train'].train_test_split(test_size=0.2)
spilt_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 30000
    })
})

In [35]:
# 2. spilt한 데이터 저장
train_dataset, validation_dataset = spilt_dataset['train'], spilt_dataset['test']
train_dataset, validation_dataset

(Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 120000
 }),
 Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 30000
 }))

In [36]:
# 3. train & validation & test split.
hf_train_dataset = train_dataset.shuffle(seed=42).select(range(5000))
hf_val_dataset = validation_dataset.shuffle(seed=42).select(range(1000))
hf_test_dataset = hf_dataset['test'].shuffle(seed=42).select(range(1000))

In [37]:
hf_train_dataset, hf_val_dataset, hf_test_dataset

(Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 5000
 }),
 Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 1000
 }),
 Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
     num_rows: 1000
 }))

### Trainer를 활용한 학습

In [38]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/Natural_Language_Processing/10_huggingface_project/transformers'

training_arguments = TrainingArguments(
    output_dir,                            # output이 저장될 경로
    evaluation_strategy="epoch",           # evaluation하는 빈도
    learning_rate = 2e-5,                  # learning_rate
    per_device_train_batch_size = 8,       # 각 device 당 batch size
    per_device_eval_batch_size = 8,        # evaluation 시에 batch size
    num_train_epochs = 1,                  # train 시킬 총 epochs
    weight_decay = 0.1,                    # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
# glue/mrpc의 acc, f1 지표 활용
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [40]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 5000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 625


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.123400,0.090472,0.980000,0.980119


Saving model checkpoint to /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=625, training_loss=0.12469301452636719, metrics={'train_runtime': 533.7338, 'train_samples_per_second': 9.368, 'train_steps_per_second': 1.171, 'total_flos': 1315555276800000.0, 'train_loss': 0.12469301452636719, 'epoch': 1.0})

### Test Set 평가

In [42]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.5503530502319336,
 'eval_accuracy': 0.898,
 'eval_f1': 0.8969696969696969,
 'eval_runtime': 36.7177,
 'eval_samples_per_second': 27.235,
 'eval_steps_per_second': 3.404,
 'epoch': 1.0}

### 결과
1. 'monologg/koelectra-base-finetuned-sentiment' pre-trained model을 활용했을 때 validation accuracy 0.98, test accuracy 0.898로 가장 양호한 결과를 보였다.
- 모델 출처 : https://huggingface.co/monologg/koelectra-base-finetuned-sentiment
2. padding은 'max_len'로 변경하여 수행하였고, weight_decay는 0.1로 설정하였다.
3. learning rate scheduler는 linear, cosine, polynomial을 각각 시도했지만 학습결과가 불안정하게 나타나면서 validation loss는 지속적으로 증가하였다. 기본값으로 학습하는 것이 가장 양호하였다.
4. batch size는 'out of memory' 문제로 인하여 8로 설정하였다.

# STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교

In [44]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.5435280799865723,
 'eval_accuracy': 0.898,
 'eval_f1': 0.89738430583501,
 'eval_runtime': 37.0446,
 'eval_samples_per_second': 26.994,
 'eval_steps_per_second': 3.374,
 'epoch': 1.0}

In [63]:
import tensorflow as tf
from transformers import TrainingArguments, TFTrainer
from transformers.data.data_collator import DataCollatorWithPadding
from transformers import AutoTokenizer

class CustomDataCollator(DataCollatorWithPadding):
    def __init__(self, tokenizer, max_length=None, padding=True, pad_to_multiple_of=None, bucketing=True):
        super().__init__(
            tokenizer=tokenizer,
            max_length=max_length,
            padding=padding,
            pad_to_multiple_of=pad_to_multiple_of
        )
        self.bucketing = bucketing

    def collate_batch(self, features):
        if self.bucketing:
            # 버킷 내에서 가장 긴 시퀀스의 길이를 찾습니다.
            max_length = max(len(feature["input_ids"]) for feature in features)
            # 버킷을 만듭니다. 길이가 10부터 110까지 10씩 증가하는 버킷을 만듭니다.
            buckets = {length: [] for length in range(10, 111, 10)}
            for feature in features:
                input_ids = feature["input_ids"]
                for bucket_size in buckets.keys():
                    if len(input_ids) <= bucket_size:
                        buckets[bucket_size].append(feature)
                        break
            
            batch = []
            # 각 버킷에 대해 동적 패딩을 적용합니다.
            for bucket_size, bucket in buckets.items():
                if not bucket:
                    continue
                batch += self.dynamic_padding(bucket, max_length)
        else:
            batch = self.dynamic_padding(features, self.max_length)
        
        return tf.data.Dataset.from_tensor_slices(batch)

    def dynamic_padding(self, features, max_length):
        # 가장 긴 시퀀스의 길이에 따라 패딩합니다.
        if self.padding:
            max_length = min(max_length, self.max_length)
            for feature in features:
                padding_length = max_length - len(feature["input_ids"])
                feature["input_ids"] = feature["input_ids"] + [self.tokenizer.pad_token_id] * padding_length
                feature["attention_mask"] = feature["attention_mask"] + [0] * padding_length
        return features

In [64]:
from transformers import TFTrainer

huggingface_model_2 = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-finetuned-sentiment", num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-finetuned-sentiment")

data_collator = CustomDataCollator(tokenizer=tokenizer, bucketing=True)

# TrainingArguments 설정
training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy="epoch",  
    learning_rate = 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    warmup_steps=500,
    weight_decay = 0.1,               
)

#dataset = data_collator.collate_batch(hf_dataset['train'].shuffle(seed=42).select(range(6000)))

# 트레이너 초기화
trainer = Trainer(
    model=huggingface_model_2,
    args=training_arguments,
    train_dataset=hf_dataset['train'].shuffle(seed=42).select(range(5000)),
    eval_dataset=hf_dataset['test'].shuffle(seed=42).select(range(1000)),
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# 학습 시작
trainer.train()

loading configuration file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a0a010ec774eac100a824caba3e4730b7d8d74824d5fd9a63fbf3e910354d3f8.235694219f1bd11aa2574b58d596fa99ecdcab54e7cd4eb291e2186b33bd124b
Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sum

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.206700,0.523911,0.905000,0.904137


Saving model checkpoint to /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=625, training_loss=0.184204931640625, metrics={'train_runtime': 534.4432, 'train_samples_per_second': 9.356, 'train_steps_per_second': 1.169, 'total_flos': 1315555276800000.0, 'train_loss': 0.184204931640625, 'epoch': 1.0})

## Bucketing과 Dynamic Padding에 대한 설명
#### Bucketing
1. Bucketing은 주어진 데이터의 시퀀스 길이에 따라 데이터를 그룹화하는 것이다. NLP모델은 보통 고정된 크기의 입력 데이터를 받지만, 실제 데이터는 길이가 다양하다. 이러한 상황에서 길이가 다른 데이터를 처리하기 위해 사용하는 방법이 Bucketing이다.

2. Bucketing 방식을 예로 들면, 문장의 길이가 10 이하인 데이터를 하나의 그룹에 할당하고, 길이가 10 ~ 20 인 데이터를 다른 그룹에 할당하는 것이다.

3. 이러한 Bucketing 방법으로 배치를 만들 때 padding을 최소화하고 메모리를 효율적으로 사용할 수 있고 학습 속도를 향상시킨다.

#### Dynamic Padding
1. Dynamic Padding은 각 배치에 포함된 시퀀스들의 길이에 따라 실제로 필요한 패딩의 양을 조절하는 것을 의미한다. Dynamic Padding을 사용하면 각 배치의 시퀀스 길이에 따라 실제로 필요한 패딩의 양이 달라진다. 

2. Dynamic Padding은 길이가 가장 긴 시퀀스에 맞춰 다른 시퀀스들을 패딩하는 것이 아니라, 각 배치에 있는 시퀀스 중 가장 길이가 긴 시퀀스의 길이만큼만 패딩을 적용한다. 

3. 이렇게 함으로써, 각 시퀀스의 실제 길이만큼만 연상을 수행하게 되어 계산 효율을 높일 수 있다.

Bucketing과 Dynamic Padding은 모두 자연어 처리 모델의 성능과 효율성을 향상시키는 데 도움이 될 수 있다.

## Step 4. 와 Step 5. 결과 비교
1. 시간 단축 - step4 에서 모델 학습에 걸린 시간은 08분 52초인 반면, Bucketing과 Dynamic Padding을 적용했을 때는 약 8분 17초가 소요되었다. Bucketing과 Dynamic Padding을 적용했을 때 시간 단축의 효과가 있는 것으로 보인다.

2. 성능 - 성능면에서는 step4의 validation이 0.98로, step5의 0.9보다 높게 나왔다. 시간 및 메모리 제한으로 인해 epoch을 1로 설정했기 때문에 epoch 수를 늘린 추가적인 실험을 시도해서 결과를 확인해야 할 필요가 있다. 

결과적으로, Bucketing task를 수행할 때 연산 속도와 모델 성능 간의 trade-off 관계가 있다면, 위 비교결과가 적절하게 나온 것으로 생각할 수 있다.

# 회고
- 부족했던 점, 추가적으로 시도 해볼만 한 것들
1. 'CUDA out of memory' 문제와 시간의 제한으로 인해 데이터와 모델, 하이퍼파라미터를 최소화할 수밖에 없었다. 만일 충분한 컴퓨팅 자원이 갖춰져 있다면, 샘플의 크기와 다양한 하이퍼파라미터 튜닝을 통해 일반적인 성능결과를 비교대조하면서 확인해 보고 싶다.

- 문제해결
1. huggingface의 다양한 라이브러리를 이번 프로젝트를 통해 처음 접했다. 'Dataset' 타입으로 생성된 데이터셋은 지금까지 다뤘던 csv 파일을 전처리하던 것과 같이 handling할 수 없었다. 단시간에 huggingface의 라이브러리를 세부적이게 구현하는 것은 분명 어려움이 있었다.

2. 따라서 fine-tuning을 통한 성능 향상에 어려움이 있었다. csv파일과 pandas 도구를 활용한 한국어 전처리를 수행하지 못하기 때문에 TrainingArguments, Trainer의 파라미터를 조정하면서 많은 시도를 해보았다. 결론부터 말하면 기존 klue base 모델로는 validation accuracy 0.888을 넘지 못했다. 추가적으로 조사해본 결과, 기존 존재하던 대부분의 pre-trained model도 89%를 거의 대부분 넘지 못했고(nsmc 태스크), 일부 Hanbret, koelectra 모델만 90%를 살짝 넘은 leaderboard를 확인하였다.

3. 결과적으로 모델을 'monologg/koelectra-base-finetuned-sentiment' pre-traind 모델로 바꿨을 때, nsmc 데이터 이진분류 태스크에서 validation accuracy가 90% 넘는 결과를 확인할 수 있었다.

- 아쉬운 점
1. Bucketing과 Dynamic Padding에 대해 설명된 학습자료가 없는 것이 아쉬웠다. 공식문서에서는 구체적인 설명을 찾을 수 없었다. 구글링과 ChatGPT를 활용해서 어찌됬든 코드를 구현하긴 했지만 제대로 된 것인지 사실 확인할 길이 없다.

In [43]:
# Bucketing과 Dynamic Padding 구현 코드 2

# from transformers import DataCollatorWithPadding
# from transformers import BertTokenizer

# tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-finetuned-sentiment")
# huggingface_model_2 = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-finetuned-sentiment", num_labels = 2)

# # 데이터를 토큰화하고 패딩을 추가하는 DataCollatorWithPadding 인스턴스 생성
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# training_arguments = TrainingArguments(
#     output_dir,                            # output이 저장될 경로
#     evaluation_strategy="epoch",           # evaluation하는 빈도
#     learning_rate = 2e-5,                  # learning_rate
#     per_device_train_batch_size = 8,       # 각 device 당 batch size
#     per_device_eval_batch_size = 8,        # evaluation 시에 batch size
#     num_train_epochs = 1,                  # train 시킬 총 epochs
#     weight_decay = 0.1,                    # weight decay
#     group_by_length=True
# )

# trainer = Trainer(
#     model=huggingface_model_2,           # 학습시킬 model
#     args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
#     train_dataset=hf_dataset['train'].shuffle(seed=42).select(range(5000)),
#     eval_dataset=hf_dataset['test'].shuffle(seed=42).select(range(1000)),
#     compute_metrics=compute_metrics,
#     data_collator=data_collator,       # 데이터 콜래이터 설정
#     #tokenizer=tokenizer
# )
# trainer.train()

loading configuration file https://huggingface.co/monologg/koelectra-base-finetuned-sentiment/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a0a010ec774eac100a824caba3e4730b7d8d74824d5fd9a63fbf3e910354d3f8.235694219f1bd11aa2574b58d596fa99ecdcab54e7cd4eb291e2186b33bd124b
Model config ElectraConfig {
  "_num_labels": 2,
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sum

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.131100,0.097968,0.979000,0.979146


Saving model checkpoint to /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/Natural_Language_Processing/10_huggingface_project/transformers/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=625, training_loss=0.13212779541015626, metrics={'train_runtime': 533.4271, 'train_samples_per_second': 9.373, 'train_steps_per_second': 1.172, 'total_flos': 1315555276800000.0, 'train_loss': 0.13212779541015626, 'epoch': 1.0})